In [1]:
# from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoTokenizer
# from transformers import TrainingArguments
# from trl import SFTTrainer
# from evaluate import load
# from peft import LoraConfig, prepare_model_for_kbit_training
from peft import PeftModel 
from args_parser import get_args
from pathlib import Path
from utils import *
from tqdm import tqdm

In [2]:
tmp=0.9
top_p=0.6
max_length=1024

args = get_args()
dataset = get_dataset(
        args.source_path, args.target_path, field="prompt", prompt_only=True
    )
    

model_name = args.model_name

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
model = PeftModel.from_pretrained(model, "experiments/Llama-2-7b-hf/checkpoint-10000")
# m = m.merge_and_unload()
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
tokenizer.pad_token = tokenizer.eos_token

Found cached dataset text (/home/daria.kotova/.cache/huggingface/datasets/text/default-ef683339c6e1d1b5/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/daria.kotova/.cache/huggingface/datasets/text/default-594e1b1157c143e0/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Loading cached processed dataset at /home/daria.kotova/.cache/huggingface/datasets/text/default-ef683339c6e1d1b5/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-f74c4750f7345492.arrow


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 
)

In [4]:
print(len(dataset))

15138


In [5]:
pad_id  = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

loss_fct = CrossEntropyLoss(reduction="mean", ignore_index=pad_id)
perplexities_loss = []
perplexities_loss_shifted = []
perplexities_model = []
nlls = []

with torch.no_grad():
    for batch in tqdm(dataloader):


        inputs = tokenizer(batch["prompt"], padding=False, return_tensors="pt").to('cuda')

        story_lengths= len(tokenizer(batch["story"], padding=False, return_tensors="pt")['input_ids'][0])

        out_logits = model(**inputs).logits
        out_logits =  out_logits[0][-story_lengths:]
        labels_inputs = inputs['input_ids'][0][-story_lengths:]
        shifted_labels = labels_inputs[1:]
        shifted_logits = out_logits[:-1]

        perplexities_loss_shifted.append(torch.exp(loss_fct(shifted_logits, shifted_labels)))

        nll = loss_fct(out_logits , labels_inputs)
        perplexity_batch = torch.exp(nll)

        nlls.append(nll)

        outputs = model(inputs["input_ids"], labels=inputs["input_ids"])
        neg_log_likelihood = outputs.loss
        # print(nll, neg_log_likelihood)

        perplexities_model.append(torch.exp(neg_log_likelihood).item())
        perplexities_loss.append(perplexity_batch.item())

        

    ppl_model = np.mean(perplexities_model)
    ppl_loss = np.mean(perplexities_loss)
    print("Final perplexity is: ", ppl_model, ppl_loss)

  0%|          | 0/15138 [01:32<?, ?it/s]


KeyboardInterrupt: 

In [6]:
a = [[1, 0, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0]]
b = [0, 2, 2, 3]

loss_fct = CrossEntropyLoss(reduction="none")

a = torch.tensor(np.array(a))
b = torch.tensor(np.array(b))

In [8]:
torch.exp(loss_fct(a.float(), b))

tensor([2.4715, 2.4715, 2.4715, 2.4715])

In [67]:
loss_fct(a.float(), b)

tensor([0.9048, 0.9048, 0.9048, 0.9048])

In [68]:
out_logits

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
